In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install langchain-openai
!pip install chromadb
!pip install pypdf

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(openai_api_key='123', organization="gluon-meson",
openai_api_base="https://0972-152-101-166-135.ngrok-free.app", model='gpt-3.5-turbo', temperature=0, streaming=False)

embeddings = OpenAIEmbeddings(openai_api_key='123', organization="gluon-meson",
openai_api_base="https://0972-152-101-166-135.ngrok-free.app", model='text-embedding-ada-002')

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="https://0972-152-101-166-135.ngrok-free.app",
    api_key="123",
)

system_prompt =  """  As an emotion analysis expert, please evaluate the sentiment of the following user review and indicate whether it is positive (Y) or negative (N):
                      If it is not clear whether the review is positive or negative, return -1
                  """

prompt = system_prompt

response = client.chat.completions.create(
    model = "gpt-3.5-turbo", # for OpenAI
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "这个东西太裤辣"},
    ],
    temperature = 0,
    max_tokens = 2000,
  )
print(response.choices[0].message.content)



-1


In [ ]:
## exercise1
from openai import OpenAI
client = OpenAI(
    base_url="https://0972-152-101-166-135.ngrok-free.app/",
    api_key="123",
)

system_prompt0 =  """  When a user enters a comment, You analyze it to determine whether it is positive or negative.
                      If it is a positive comment, output Y; if it is a negative comment, output N.
                      If the sentiment of the comment cannot be identified, -1 is output.
                  """

system_prompt =  """  When a user enters a comment, you analyze it, determine the sentiment of the comment,
                      and accurately describe it with an emotion word.
                      If the sentiment of the comment cannot be identified, -1 is output.
                  """

prompt = system_prompt

response = client.chat.completions.create(
    model = "gpt-3.5-turbo", # for OpenAI
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "花谢了"},
    ],
    temperature = 0,
    max_tokens = 2000,
  )
print(response.choices[0].message.content)


-1


In [ ]:
# AI Agent

In [ ]:
import requests
import json
from pydantic import BaseModel, Field


class Addition(BaseModel):
    """Get the summation"""
    argument1: float = Field(description="number, Only keep 2 decimal places")
    argument2: float = Field(description="number, Only keep 2 decimal places")

def get_summation(argument1:float, argument2:float) -> float:
    return argument1 + argument2


get_summation(30.57, 104.07)

134.64

In [ ]:
from pydantic import BaseModel, Field

class Multiplication(BaseModel):
  """Call this to get the result of multiplying two numbers"""
  multiplier: float = Field(description="number, Only keep 2 decimal places")
  multiplicand: float = Field(description="number, Only keep 2 decimal places")

def multiply(multiplier: float, multiplicand: float) -> float:
  return multiplier * multiplicand

multiply(1.0,9.0)

9.0

In [ ]:
from pydantic import BaseModel, Field

class PowerCalculator(BaseModel):
    """Calculator to compute the power of a number"""

    base: float = Field(description="The base number to be raised to a power, only keep 2 decimal places")
    exponent: float = Field(description="The exponent to which the base is raised, only keep 2 decimal places")

def calculate_power( base: float, exponent: float) -> float:
    """Calculate the power of the base number raised to the exponent"""
    return base ** exponent

# 使用示例
# 创建一个PowerCalculator实例并计算2的3次方
# calculator = PowerCalculator(base=2, exponent=3)
result = calculate_power(2, 3)
print(f"The result  {result}")

The result  8


In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import  StructuredTool

agent_tools=[StructuredTool.from_function(func=get_summation, name="Addition", description="Get the summation"),
             StructuredTool.from_function(func=multiply, name="Multiplication", description="Call this to get the result of multiplying two numbers"),
       StructuredTool.from_function(func=calculate_power, name="PowerCalculator", description="Calculator to compute the power of a number")]

agent = create_openai_tools_agent(llm, agent_tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=agent_tools, verbose=True)
result = agent_executor.invoke({"input":"Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `PowerCalculator` with `{'base': 3, 'exponent': 5}`


243.0
Invoking: `Addition` with `{'argument1': 12, 'argument2': 3}`


15.0
Invoking: `Multiplication` with `{'multiplier': 243, 'multiplicand': 15}`


3645.0
Invoking: `PowerCalculator` with `{'base': 3645, 'exponent': 2}`


13286025.0The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025. 🎉

> Finished chain.
The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025. 🎉
